## Bag of words mode
- each sentence should be a document
- sub-documents for each file
- heirachy:
1. subject folder (corpus)
2. sub-topics (corpus)
3. files (corpus)
4. documents (sentences (or 3) within the files)

In [308]:
import string

from urllib import request
from urllib.request import FancyURLopener

import nltk
from nltk.corpus import stopwords
from nltk import word_tokenize , sent_tokenize
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.util import ngrams , bigrams
from nltk.lm import MLE
from nltk.lm.preprocessing import pad_both_ends
from nltk import SnowballStemmer
from nltk.stem.porter import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer

import bs4 as bs
from bs4 import BeautifulSoup as bsoup
from bs4 import SoupStrainer

# nltk.download('universal_tagset')
# nltk.download('wordnet')
# nltk.download('averaged_perceptron_tagger')
# nltk.download('stopwords')
import scipy
from scipy import spatial

import numpy
from numpy import dot

import sqlite3
connection = sqlite3.connect('database.db')
import warnings
warnings.filterwarnings('ignore')


from sklearn . model_selection import train_test_split
from sklearn . feature_extraction . text import CountVectorizer
from sklearn . feature_extraction . text import TfidfTransformer
from sklearn . linear_model import LogisticRegression

import os

In [309]:
subject = "medicine"
def find_topic(subject):
    path = 'Datasets\\topic_list\\' + subject
    for file in os.listdir(path):
        with open(path, encoding='utf-8', errors='ignore', mode='r') as document:
            content = document.read()
            topic = document.name
        print(topic)
        



def process_text(query):
    #tokenization
    tokenized_query = word_tokenize(query)

    #removing stop words
    #maybe make quotes a search prompt
    tokens_wo_sw= [word for word in tokenized_query if not word in stopwords.words()]
    filtered_query = ("").join(tokens_wo_sw)

    postmap = {
    'ADJ': 'j',
    'ADV': 'r',
    'NOUN': 'n',
    'VERB': 'v'
    }
    
    #lemmentised and tagged query
    lemm_q = []
    lemmentiser = WordNetLemmatizer()
    tagged_query = nltk.pos_tag(lemm_q, tagset='universal')
    for token in tagged_query: 
        word = token
        tag = token[0]
        tag = token[1]
        if tag in postmap.keys():
            lemm_q.append(lemmentiser.lemmatize(word, postmap[tag]))
        else:
            lemm_q.append(lemmentiser.lemmatize(word))

    return lemm_q
    # #only return false if no similar paper is found
    # return falleback_response(query)
    

In [310]:
#how long should each passage be?
documents = {
"History": "Datasets\\battle_of_hastings.txt",
"Medicine": "Datasets\\biasesinnlp.txt",
"ComputerScience": "Datasets\\brain-injury.txt"
}

topic = ""
from sklearn.feature_extraction.text import CountVectorizer
corpus = {}
for subject in documents:
    with open(documents.get(subject), encoding="utf8", errors="ignore") as f:
        corpus[subject] = f.read()


all_text = corpus.values()

#binary = 0 because we arent interested in text where the keywords are not there
count_vect = CountVectorizer(stop_words=stopwords.words('english'), binary=0, ngram_range=(1,3))
X_train_counts = count_vect.fit_transform(all_text)

In [311]:
from sklearn . feature_extraction . text import TfidfTransformer

count_vect = CountVectorizer( stop_words = stopwords.words('english'))
X_train_counts = count_vect.fit_transform ( all_text )
tf_transformer = TfidfTransformer ( use_idf = True , sublinear_tf = True ). fit (X_train_counts )
X_train_tf = tf_transformer.transform( X_train_counts )
# print ( X_train_tf.shape )



lemmentiser = WordNetLemmatizer()
analyser = CountVectorizer().build_analyzer()

def lemmed_words (text):
    tokenized_query = word_tokenize(text)
    tokens_wo_sw= [word.lower() for word in tokenized_query if not word in stopwords.words()]
    # print(tokens_wo_sw)
    
    postmap = {
    'ADJ': 'j',
    'ADV': 'r',
    'NOUN': 'n',
    'VERB': 'v'
    }
    
    #lemmentised and tagged query
    lemm_q = []
    lemmentiser = WordNetLemmatizer()
    tagged_query = nltk.pos_tag(tokens_wo_sw, tagset='universal')
    for token in tagged_query: 
        word = token[0]
        tag = token[1]
        if tag in postmap.keys():
            lemm_q.append(lemmentiser.lemmatize(word, postmap[tag]))
        else:
            lemm_q.append(lemmentiser.lemmatize(word))
    
    return lemm_q

lemm_vectorizer = CountVectorizer( analyzer = lemmed_words )
print ( lemm_vectorizer.fit_transform(['This sentence should get seriously mangled in the stemming process, but it is fine.']) )
print(lemm_vectorizer.get_feature_names ())


  (0, 7)	1
  (0, 5)	1
  (0, 3)	1
  (0, 6)	1
  (0, 4)	1
  (0, 0)	1
  (0, 2)	1
  (0, 1)	1
[',', '.', 'fine', 'mangle', 'process', 'sentence', 'stem', 'this']


## Training and testing the chatbot
- classifying parts of text
- maybe should be able to add new classess?

In [312]:
from sklearn.model_selection import train_test_split

label_dir = {
"history": "Datasets/training/history",
"medicine": "Datasets/training/medicine",
"greetings": "Datasets/training/greetings"
}

data = []
labels = []

for label in label_dir . keys () :
    for file in os . listdir ( label_dir [ label ]) :
        filepath = label_dir [ label ] + os . sep + file
        with open(filepath , encoding ='utf8', errors ='ignore', mode ='r') as review:
            content = review . read ()
            data . append ( content )
            labels . append ( label )

X_train , X_test , y_train , y_test = train_test_split ( data , labels , stratify = labels, test_size =0.25 , random_state =1)

count_vect = CountVectorizer (stop_words = stopwords . words ('english'))
X_train_counts = count_vect . fit_transform ( X_train )

tfidf_transformer = TfidfTransformer ( use_idf = True , sublinear_tf = True ). fit (
X_train_counts )
X_train_tf = tfidf_transformer . transform ( X_train_counts )

classifier = LogisticRegression ( random_state =0) . fit ( X_train_tf , y_train )


## Bag of words and training

In [313]:
from sklearn.metrics import accuracy_score , f1_score , confusion_matrix

data = []
labels = []
filepath = "Datasets/training/"
for label in label_dir . keys () :
    for file in os . listdir ( label_dir [ label ]) :
        filepath = label_dir [ label ] + os . sep + file
        with open(filepath , encoding ='utf8', errors ='ignore', mode ='r') as review:
            content = review.read()
            data.append (content )
            labels.append ( label )

X_train, X_test, y_train, y_test = train_test_split(data, labels, stratify=labels, test_size=0.25, random_state=42)
print(X_train, y_train)

['how many bones are in the human body?', 'hello and good morning!', 'who was the first emperor of ancient rome?', 'when was the battle of hastings?', 'hi! hello! how are you?', 'greetings! how are you today?', 'what are the effects of smoking on the lungs?'] ['medicine', 'greetings', 'history', 'history', 'greetings', 'greetings', 'medicine']


In [314]:
from nltk.corpus import stopwords
from nltk.stem.snowball import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer

p_stemmer = PorterStemmer()
analyzer = CountVectorizer().build_analyzer()

def stemmed_words(doc):
    return (p_stemmer.stem(w) for w in analyzer(doc))

count_vect = CountVectorizer(lowercase=True, stop_words=stopwords.words('english'), analyzer=stemmed_words)
X_train_counts = count_vect.fit_transform(X_train)

print(X_train_counts)

  (0, 14)	1
  (0, 18)	1
  (0, 5)	1
  (0, 2)	1
  (0, 16)	1
  (0, 24)	1
  (0, 15)	1
  (0, 4)	1
  (1, 12)	1
  (1, 1)	1
  (1, 9)	1
  (1, 19)	1
  (2, 24)	1
  (2, 29)	1
  (2, 26)	1
  (2, 8)	1
  (2, 7)	1
  (2, 20)	1
  (2, 0)	1
  (2, 22)	1
  (3, 24)	1
  (3, 26)	1
  (3, 20)	1
  (3, 28)	1
  (3, 3)	1
  (3, 11)	1
  (4, 14)	1
  (4, 2)	1
  (4, 12)	1
  (4, 13)	1
  (4, 30)	1
  (5, 14)	1
  (5, 2)	1
  (5, 30)	1
  (5, 10)	1
  (5, 25)	1
  (6, 2)	1
  (6, 24)	2
  (6, 20)	1
  (6, 27)	1
  (6, 6)	1
  (6, 23)	1
  (6, 21)	1
  (6, 17)	1


In [315]:

from sklearn.feature_extraction.text import TfidfTransformer

tfidf_transformer = TfidfTransformer(use_idf=True, sublinear_tf=True).fit(X_train_counts)
X_train_tf = tfidf_transformer.transform(X_train_counts)

print(X_train_tf)

  (0, 24)	0.2461638358943208
  (0, 18)	0.3996040294183947
  (0, 16)	0.3996040294183947
  (0, 15)	0.3996040294183947
  (0, 14)	0.28353100387524055
  (0, 5)	0.3996040294183947
  (0, 4)	0.3996040294183947
  (0, 2)	0.2461638358943208
  (1, 19)	0.5206467559864713
  (1, 12)	0.43218152024617124
  (1, 9)	0.5206467559864713
  (1, 1)	0.5206467559864713
  (2, 29)	0.3900791205824988
  (2, 26)	0.323799169804132
  (2, 24)	0.24029630723351017
  (2, 22)	0.3900791205824988
  (2, 20)	0.27677279633666213
  (2, 8)	0.3900791205824988
  (2, 7)	0.3900791205824988
  (2, 0)	0.3900791205824988
  (3, 28)	0.4676802387867398
  (3, 26)	0.3882147622431314
  (3, 24)	0.2881001017914568
  (3, 20)	0.3318331093628171
  (3, 11)	0.4676802387867398
  (3, 3)	0.4676802387867398
  (4, 30)	0.459671713122356
  (4, 14)	0.39291214216113446
  (4, 13)	0.553764043681409
  (4, 12)	0.459671713122356
  (4, 2)	0.34112939594570285
  (5, 30)	0.43920778363284724
  (5, 25)	0.5291112577469715
  (5, 14)	0.3754202536171529
  (5, 10)	0.529111257

In [316]:
from sklearn.linear_model import LogisticRegression

clf = LogisticRegression(random_state=0).fit(X_train_tf, y_train)

## Evaluating the classifier

In [317]:
# Preprocessing documents and creating term-document matrix
X_new_counts = count_vect.transform(X_test)

# Weighting
X_new_tfidf = tfidf_transformer.transform(X_new_counts)

# Predict on the test set
predicted = clf.predict(X_new_tfidf)

# Print metrics
print(confusion_matrix(y_test, predicted))
print(accuracy_score(y_test, predicted))
print(f1_score(y_test, predicted, pos_label='greetings', average='weighted'))

[[1 0 0]
 [0 1 0]
 [1 0 0]]
0.6666666666666666
0.5555555555555555
